<a href="https://colab.research.google.com/github/Sunday-Okey/NLP_Project_Machine_Translation/blob/master/NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB:
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"


if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

In [2]:
pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os

In [4]:
# Download the data
import urllib.request
datapath = os.path.join("data", "")
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/Sunday-Okey/NLP_Project_Machine_Translation/master"
os.makedirs(datapath, exist_ok=True)
for filename in ("small_vocab_en", "small_vocab_fr"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + "/data/" + filename
    urllib.request.urlretrieve(url, datapath + filename)

In [5]:
# Download project files
import urllib.request
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/Sunday-Okey/NLP_Project_Machine_Translation/master/"
for filename in ("helper.py", "project_tests.py"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + filename
    urllib.request.urlretrieve(url, filename)

In [6]:
# import the libraries
import collections
import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from tensorflow.keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from sklearn.model_selection import train_test_split

In [7]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')
print(type(english_sentences))

Dataset Loaded
<class 'list'>


Let's begin by investigating the dataset that will be used to train and evaluate our pipeline

## Files

Each line in small_vocab_en contains an English sentence with the respective translation in each line of small_vocab_fr. View the first two lines from each file.

In [8]:
for sample in range(2):
    print(f'small_vocab_en Line {sample + 1}:  {english_sentences[sample]}')
    print(f'small_vocab_fr Line {sample + 1}:  {english_sentences[sample]}')

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  the united states is usually chilly during july , and it is usually freezing in november .


From looking at the sentences, we can see they have been preprocessed already. The puncuations have been replaced by spaces. All the text have been converted to lowercase.